In [ ]:

import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup 
import os
import shutil
import glob
import yaml
import torch
!pip install easyocr -qq
import easyocr
import cv2 as cv
import matplotlib.pyplot as plt
import easyocr
%matplotlib inline
import pickle 
import cv2

# 1.  *Initialize The Preprocessing Function*
> 

In [ ]:

def normalized_coordinates(filename, width, height, xmin, ymin, xmax, ymax):
    """Take in image coordinates (unnormalized) as input, return normalized values 
    """
    
    xmin, xmax = xmin / width, xmax / width
    ymin, ymax = ymin / height, ymax/ height

    width = xmax-xmin
    height = ymax-ymin
    x_center = xmin + (width / 2)
    y_center = ymin + (height / 2)

    return x_center, y_center, width, height

def write_label(filename, x_center, y_center, width, height):
    """Save image's coordinates in text file named "filename"
    """
    with open(filename, mode='w') as outf:
        outf.write(f"{0} {x_center} {y_center} {width} {height}\n")

def parse_xml_tags(data):
    """Parse xml label file, return image file name, and its coordinates as a dictionary
    """
    tags = ['filename', 'width', 'height', 'xmin', 'ymin', 'xmax', 'ymax']
    Bs_data = BeautifulSoup(data, "xml")
    d = dict()

    for t in tags:
        text = Bs_data.find(t).text
        if all(c.isdigit() for c in text):
            d[t] = int(text)
        else:
            d[t] = text
    return d

def build_data(dir_folder, ann_file_list, img_dir):
    """Write xml labels to text file with specifications format, save at 'labels' folder.
        Move image to 'images' folder
    """
    images_folder = f"{dir_folder}/images"
    labels_folder = f"{dir_folder}/labels"
    
    os.makedirs(images_folder, exist_ok = True)
    os.makedirs(labels_folder, exist_ok = True)


    for ann_file in ann_file_list:
        with open(ann_file, 'r') as f:
            label = parse_xml_tags(f.read())
         
        img_file_name = label['filename']
        x_center, y_center, width, height = normalized_coordinates(**label)
         
        # save at 'labels' folder
        write_label(f"{labels_folder}/{img_file_name.split('.')[0]}.txt", x_center, y_center, width, height)
         
         # Move image to 'images' folder
        shutil.copy(f"{img_dir}/{img_file_name}", f"{images_folder}/{img_file_name}")

# 2.  **Loading Dataset images and it's annotation **

In [ ]:
dir_folder = "/kaggle/working/plate_datasets"

ann_list = glob.glob('/kaggle/input/car-plate-detection/annotations/*')
build_data(dir_folder, ann_list, "/kaggle/input/car-plate-detection/images")

ann_list = glob.glob('/kaggle/input/number-plate-detection/images/*.xml')
build_data(dir_folder, ann_list, "/kaggle/input/number-plate-detection/images")

In [ ]:

data = {
    "path": dir_folder,
    "train": "images",
    "val": "",
    "names": {0: "car_lisence_plate"}
    }

with open('/kaggle/working/plate_datasets/dataset.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

In [ ]:
assert len(os.listdir("/kaggle/working/plate_datasets/labels")) == len(os.listdir("/kaggle/working/plate_datasets/images"))

  # 3.  **Downloading Yolov5**

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt

In [ ]:
!python train.py --img 640 --batch 16 --epochs 15 --data /kaggle/working/plate_datasets/dataset.yaml --weights yolov5s.pt --cache ram

In [ ]:
import matplotlib.pyplot as plt

results = plt.imread('/kaggle/working/yolov5/runs/train/exp/confusion_matrix.png')
plt.imshow(results)
plt.show()


In [ ]:
from IPython.display import Image, display

# Specify the file path
image_path = '/kaggle/working/yolov5/runs/train/exp/results.png'

# Display the image
display(Image(filename=image_path))


In [ ]:
from IPython.display import Image, display

# Specify the file path
img_coo = '/kaggle/working/yolov5/runs/train/exp/labels_correlogram.jpg'

# Display the image
display(Image(filename=img_coo))

In [ ]:
from IPython.display import Image, display

# Specify the file path
img_coo = '/kaggle/working/yolov5/runs/train/exp/labels.jpg'

# Display the image
display(Image(filename=img_coo))

4. # **Training Our Model**

In [ ]:

yolo = torch.hub.load('ultralytics/yolov5', 'custom', path='/kaggle/working/yolov5/runs/train/exp/weights/best.pt')

# 5.  **Reading The plate Number**

In [ ]:
def read_plate_number(results, frame, reader):
    n = len(results)
    x_shape, y_shape = frame.shape[1], frame.shape[0]

    for i in range(n):
        row = results[i]  # Assuming results is a list of coordinates
        confidence_index = -1  # Adjust this index based on your actual data structure

        if row[confidence_index] >= 0.5:  # Take img with 0.5 confidence
            xmin, ymin, xmax, ymax = row[:4]
            plate = frame[int(ymin):int(ymax), int(xmin):int(xmax)]

            gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)
            blurred = cv.bilateralFilter(gray, 17, 15, 15)
            text = reader.readtext(blurred)
            text = ' '.join([t[1] for t in text])

            plot_img = frame.copy()

            cv.rectangle(plot_img, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)  # BBox
            cv.rectangle(plot_img, (int(xmin), int(ymin - 20)), (int(xmax), int(ymin)), (0, 255, 0), -1)  # Text label background
            #final_img = cv.putText(plot_img, f"{text}", (int(xmin), int(ymin)), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            return plot_img ,text  



In [ ]:
import cv2

In [ ]:
reader = easyocr.Reader(['en'])

image_directory = '/kaggle/input/fifty-states-car-license-plates-dataset/Fifty States License Plates/'

image_files = [os.path.join(image_directory, file) for file in os.listdir(image_directory) if file.endswith(('.jpg', '.png', '.jpeg'))]

all_coordinates = np.empty((0, 4), dtype=float)  # Assuming each coordinate has 4 values (x_min, y_min, x_max, y_max)
final_imgggg=[]
plate_img= []
text2=[]
for img_path in image_files:
    results = yolo(img_path)

    current_coordinates = results.xyxy[0][:, :-1]

    current_coordinates_cpu = current_coordinates.cpu().numpy()

    if current_coordinates_cpu.shape[1] == 5:
        current_coordinates_cpu = current_coordinates_cpu[:, :-1]  # Assuming the last column is not needed

    frame = cv2.imread(img_path)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    final_imgggg,predict_text=read_plate_number(current_coordinates_cpu, frame, reader)
    text2.append(predict_text)
    plate_img.append(final_imgggg)

# 6.    **Testing Our Model on Other Dataset**

In [ ]:
fig, axs = plt.subplots(5, 4, figsize=(18, 18))

for n, i in enumerate(list(np.random.randint(0, len(plate_img), 20))):
    plt.subplot(5, 4, n + 1)
    plt.imshow(plate_img[i])
    plt.axis('off')
    plt.title(text2[i])

plt.show()

In [ ]:
%matplotlib inline


In [ ]:
text2

In [ ]:
results


In [ ]:
torch.save(yolo.state_dict(), 'yolo_car_plate.pt')


In [ ]:
yolo2 = torch.hub.load('ultralytics/yolov5', 'custom', path='/kaggle/working/yolov5/runs/train/exp/weights/best.pt',force_reload=True)


In [ ]:
yolo.type

In [ ]:
yolo2 

In [ ]:
img=('/kaggle/input/fifty-states-car-license-plates-dataset/Fifty States License Plates/Nebraska.jpg')

In [ ]:
result222=yolo2(img)


In [ ]:
result222

In [ ]:
def read_plate_number_image(results, frame, reader):
    # Assuming results is a list of coordinates for one photo
    if len(result222) > 0:  # Check if the list is not empty
        row = result222[0]  # Take the first and only row
        confidence_index = -1  # Adjust this index based on your actual data structure

        if row[confidence_index] >= 0.5:  # Take img with 0.5 confidence
            xmin, ymin, xmax, ymax = row[:4]
            plate = frame[int(ymin):int(ymax), int(xmin):int(xmax)]

            gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)
            blurred = cv.bilateralFilter(gray, 17, 15, 15)
            text = reader.readtext(blurred)
            text = ' '.join([t[1] for t in text])

            plot_img = frame.copy()

            cv.rectangle(plot_img, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)  # BBox
            cv.rectangle(plot_img, (int(xmin), int(ymin - 20)), (int(xmax), int(ymin)), (0, 255, 0), -1)  # Text label background
            #final_img = cv.putText(plot_img, f"{text}", (int(xmin), int(ymin)), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            return plot_img ,text  
    else:
        return None, None  # No plate detected



In [ ]:
final_imgggg=[]
plate_img= []
text2=[]

current_coordinates = result222.xyxy[0][:, :-1]

current_coordinates_cpu = current_coordinates.cpu().numpy()

if current_coordinates_cpu.shape[1] == 5:
    current_coordinates_cpu = current_coordinates_cpu[:, :-1]  # Assuming the last column is not needed

frame = cv2.imread(img)
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
final_imgggg,predict_text=read_plate_number_image(current_coordinates_cpu, frame, reader)



In [ ]:
result222

In [ ]:
plate_img

In [ ]:
results22 = yolo('/kaggle/input/fifty-states-car-license-plates-dataset/Fifty States License Plates/Alabama.jpg')

In [ ]:
results22

In [ ]:
results222

In [ ]:
plt.imshow(plate_img[0])
plt.axis('off')
plt.title(text2[0])
plt.show()